In [ ]:
!pip install git+https://github.com/Sonictherocketman/gzip-classifier
from gzip_classifier import Classifier, ParallelClassifier

  Cloning https://github.com/Sonictherocketman/gzip-classifier to /tmp/pip-req-build-b4kp57gu
  Running command git clone --filter=blob:none --quiet https://github.com/Sonictherocketman/gzip-classifier /tmp/pip-req-build-b4kp57gu
  Resolved https://github.com/Sonictherocketman/gzip-classifier to commit a5e8579a2f5d3b8c9006ef1cebbe1e07a0907ff1
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install stop-words
from stop_words import get_stop_words

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re
from sklearn.model_selection import StratifiedKFold

In [ ]:
df = pd.read_csv("drive/MyDrive/HOPE2024/training_t1_sp.csv")
val = pd.read_csv("drive/MyDrive/HOPE2024/Task 1 - Validation set .csv")
test = pd.read_csv("drive/MyDrive/HOPE2024/Task 1 - Test set No label.csv")
test['category'] = ''

In [ ]:
sw = get_stop_words('spanish')
def clean_text(text):
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs

    html=re.compile(r'<.*?>')

    text = html.sub(r'',text) #Removing html tags

    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations

    text = [word.lower() for word in text.split() if word.lower() not in sw]

    text = " ".join(text) #removing stopwords

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis

    return text

In [ ]:
#df['text'] = df['text'].apply(lambda x: clean_text(x))
#val['text'] = val['text'].apply(lambda x: clean_text(x))
#test['text'] = test['text'].apply(lambda x: clean_text(x))

In [ ]:
tweets = np.concatenate([df.text.values,val.text.values])
labels = np.concatenate([df.category.values,val.category.values])




test_tweets = test.text.values

In [ ]:
for k in range(12,31):

  accuracy = 0
  total = 0
  skf = StratifiedKFold(n_splits=4, random_state=None, shuffle=False)
  for i, (train_index, test_index) in enumerate(skf.split(tweets, labels)):
    classifier = Classifier()
    classifier.train(tweets[train_index], labels[train_index])
    val_tweets = tweets[test_index]
    val_labels = labels[test_index]
    for i in range(len(val_tweets)):
      pred, count, ls = classifier.classify(val_tweets[i],k=k, include_all=True)
      if pred == val_labels[i]:
        accuracy += 1
      total += 1
  print(k,accuracy,total,accuracy/total)

1 1072 1600 0.67
2 1072 1600 0.67
3 1135 1600 0.709375
4 1130 1600 0.70625
5 1115 1600 0.696875
6 1136 1600 0.71
7 1138 1600 0.71125
8 1143 1600 0.714375
9 1136 1600 0.71
10 1137 1600 0.710625
11 1132 1600 0.7075
12 1157 1600 0.723125
13 1144 1600 0.715
14 1141 1600 0.713125
15 1137 1600 0.710625
16 1140 1600 0.7125
17 1146 1600 0.71625
18 1140 1600 0.7125
19 1141 1600 0.713125
20 1145 1600 0.715625
21 1142 1600 0.71375
22 1148 1600 0.7175
23 1139 1600 0.711875
24 1143 1600 0.714375
25 1140 1600 0.7125
26 1148 1600 0.7175
27 1141 1600 0.713125
28 1145 1600 0.715625
29 1150 1600 0.71875


In [ ]:
for index, row in test.iterrows():
  pred, count, ls = classifier.classify(clean_text(row['text']),k=13, include_all=True)
  test.at[index,'category'] = pred

In [ ]:
test[['id','category']].to_csv('predictions.csv',index=False)